## Getting Started

- Kaggle Competition - [siim-acr-pneumothorax-segmentation](https://www.kaggle.com/c/siim-acr-pneumothorax-segmentation/overview)
- Download Data 
- Create a Pytorch Dataset and Dataloader.


### Prepare data

1. Download Data
2. Extract Data


In [1]:
!kaggle datasets download -d seesee/siim-train-test

100%|██████████████████████████████████████| 1.92G/1.92G [02:15<00:00, 24.3MB/s]
100%|██████████████████████████████████████| 1.92G/1.92G [02:15<00:00, 15.2MB/s]


In [2]:
!unzip -q siim-train-test.zip

In [3]:
ls siim/

dicom-images-test/  dicom-images-train/  train-rle.csv


In [11]:
from fastai.vision.all import *
import pydicom

In [4]:
data_path = Path('siim/')

In [5]:
data_path.ls()

(#3) [Path('siim/dicom-images-test'),Path('siim/dicom-images-train'),Path('siim/train-rle.csv')]

In [79]:
train_meta = pd.read_csv(data_path/'train-rle.csv')
train_meta.columns = [o.strip() for o in train_meta.columns]

In [80]:
train_meta.head(2)

,ImageId,EncodedPixels
0,1.2.276.0.7230010.3.1.4.8323329.6904.1517875201.850819,-1
1,1.2.276.0.7230010.3.1.4.8323329.13666.1517875247.117800,557374 2 1015 8 1009 14 1002 20 997 26 990 32 985 38 980 42 981 42 979 43 978 45 976 47 964 59 956 66 925 98 922 101 917 106 916 106 916 107 914 109 909 113 907 116 904 118 903 120 902 120 902 121 900 122 899 124 898 124 898 125 897 125 898 125 896 126 895 127 895 128 895 128 895 128 894 128 895 128 895 128 895 128 895 128 895 128 894 130 893 130 893 130 893 130 893 129 894 129 894 129 894 129 895 127 897 126 898 126 898 125 898 126 898 125 899 125 899 125 899 124 900 124 900 125 899 125 899 125 899 125 899 126 898 127 897 128 897 128 896 129 895 130 894 132 892 133 891 134 890 136 888 137...


In [28]:
train_path = (data_path/'dicom-images-train/')
train_files = list(train_path.glob('*/*/*.dcm'))

In [29]:
train_files[0]

Path('siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.1000.1517875165.878026/1.2.276.0.7230010.3.1.3.8323329.1000.1517875165.878025/1.2.276.0.7230010.3.1.4.8323329.1000.1517875165.878027.dcm')

In [32]:
o = pydicom.dcmread(train_files[0])

In [31]:
Image

<module 'PIL.Image' from '/opt/conda/lib/python3.6/site-packages/PIL/Image.py'>

In [38]:
img = Image.fromarray(o.pixel_array)

In [50]:
img_path = (data_path/'train/images')
mask_path = (data_path/'train/mask')

img_path.mkdir(parents=True, exist_ok=True)
mask_path.mkdir(parents=True, exist_ok=True)

In [52]:
ls siim/train/images/

1.2.276.0.7230010.3.1.4.8323329.1000.1517875165.878027.png


In [46]:
id = train_files[0].stem

In [48]:
img.save(f'{data_path}/train/images/{id}.png')

In [53]:
mask = 

SyntaxError: invalid syntax (<ipython-input-53-51cf80055bfa>, line 1)

In [61]:
def save_dcm_to_png(file,dst_path):
    """Extracts image from DCM file and saves it as a PNG file."""
    fname = file.stem
    pyds = pydicom.dcmread(file)
    img = Image.fromarray(pyds.pixel_array)
    img.save(f'{dst_path}/{fname}.png')
    

In [62]:
save_dcm_to_png(train_files[1],data_path/'train/images/')

In [102]:
for i in range(100):
    fname = train_files[i].stem 
    if train_meta[train_meta['ImageId']==fname]['EncodedPixels'].values[0] != '-1':
        print(i)
        break

6


In [104]:
fname = train_files[6].stem
train_meta[train_meta['ImageId']==fname]['EncodedPixels']

4919    209126 1 1019 6 1015 10 1012 13 1010 14 1008 16 1007 16 1006 18 1004 20 1003 20 1002 22 1001 22 1001 23 999 24 999 24 999 25 997 26 997 27 996 27 996 27 996 28 995 28 995 28 995 29 994 29 994 30 993 30 993 30 993 31 992 31 992 31 992 32 991 32 992 32 991 33 990 33 990 33 991 33 990 33 990 33 990 34 990 33 990 34 989 35 988 35 989 35 988 35 989 35 988 35 989 34 989 35 989 34 989 34 989 35 989 34 989 35 988 36 988 35 988 35 989 35 989 34 989 34 990 34 989 34 990 34 989 34 990 34 989 34 990 34 990 34 989 34 990 33 990 34 990 33 991 32 991 33 991 32 992 32 991 33 991 32 992 32 991 32 992 32 99...
Name: EncodedPixels, dtype: object

In [94]:
fname = train_files[3].stem

In [95]:
fname

'1.2.276.0.7230010.3.1.4.8323329.10002.1517875220.939397'

In [96]:
train_meta.columns

Index(['ImageId', 'EncodedPixels'], dtype='object')

In [99]:
train_meta[train_meta['ImageId']==fname]['EncodedPixels'].values[0]

'-1'